In [ ]:
import time
import re
import sqlite3
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By 
import csv
import pandas as pd

# 예외 처리를 위한 모듈
from selenium.webdriver.support import expected_conditions as EC

# 익명의 사용자 계정 들어가기
def main():

    
    with open('address.txt', "r", encoding="utf-8") as f:
        addressList = []
        for i in f.readlines():
            i = i.replace('\ufeff', '')
            i = i.replace('\n', '')
            addressList.append(i)
            print('ccc')

        for j in range(len(addressList)):
            
            driver = webdriver.Chrome("C:/driver/chromedriver.exe")
            address = addressList[j]
            url = 'https://watcha.com/en-KR/users/'+address+'/contents/books/ratings'
            print('url주소 : ', url)
            driver.get(url)
            print('ddd')
            time.sleep(5)
            
            # 스크롤 제일 끝으로 내리기
            last_height = driver.execute_script("return document.body.scrollHeight")
            

            while True:
                # Scroll down to bottom
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait to load page
                time.sleep(2)

                # Calculate new scroll height and compare with last scroll height
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height


                wTitle = []
                wAuthor = []
                wRating = []
                
                for j in range(1,3):
                    # 도서 상세 페이지 들어가기
                    findurl = (driver.find_elements_by_xpath('//*[@id="root"]/div/div/section/section/div[1]/section/div[1]/div/ul/li[*]/a'))[j].get_attribute('href')
                    driver.get(findurl)
                    time.sleep(3)
                    
                    # 책 제목, 저자, 평점 가져와서 dataframe에 담기
                    title = driver.find_element_by_xpath('//*[@id="root"]/div/div/section/div/div/div/section/div[2]/div/div/div/div/h1').text
                    author = driver.find_element_by_xpath('//*[@id="root"]/div/div/section/div/div/div/section/div[2]/div/div/div/div/div[1]').text
                    brating = driver.find_element_by_xpath('//*[@id="root"]/div/div/section/div/div/div/section/div[2]/div/div/div/div/div[2]').text
                    rating = brating[8:11]
                
                    wTitle.append(title)
                    wAuthor.append(author)
                    wRating.append(rating)
                    driver.back()
                    
                df01 = pd.DataFrame(wTitle)
                df02 = pd.DataFrame(wAuthor)
                df03 = pd.DataFrame(wRating)
                df04 = pd.concat([df01, df02, df03], axis = 1)

            # 사용자별로 cvs 파일 생성
            
                df04.to_csv('unknown{}.csv'.format(i), index = False, header=None, encoding = 'utf-8-sig')

            time.sleep(3)
            driver.quit()
            print('eee')
               # time.sleep(13)

                
                
#time.sleep(3)
if __name__ == "__main__":
    main()